In [2]:
import os
import pandas as pd

In [3]:
import os

folder = "/home/trishalaswain/Downloads/SCP_2474/"
# folder = "/home/trishalaswain/Downloads/Feb_01_to_28_tsv (1)/"
dfs = []
for filename in os.listdir(folder):
        if filename.endswith("15.xlsx") or filename.endswith("28.xlsx"):
            print(os.path.join(folder, filename))
            dfs.append(pd.read_excel(os.path.join(folder, filename),index_col=False))
            # break
df_data = pd.concat(dfs)

/home/trishalaswain/Downloads/SCP_2474/SCP-2474-OCHSNER MEDICAL CENTER - WEST BANK-2023-02-01-2023-02-28.xlsx
/home/trishalaswain/Downloads/SCP_2474/SCP-2474-OCHSNER MEDICAL CENTER - WEST BANK-2023-03-01-2023-03-15.xlsx


In [29]:
# df_data = pd.read_excel("/home/jaiminrana/Analysis/SCP/SCP-3412-BAPTIST MEMORIAL HOSPITAL - ATTALA-2023-03-01-2023-03-15.xlsx")
df_use = df_data[['id','EANDM_Suggested','EANDM_Final','CPT_Final']]

In [30]:
df_use = df_use[(df_use.EANDM_Final.str.contains('99284')) | (df_use.EANDM_Final.str.contains('99285'))]
# df_use.set_index('accountNumber',inplace = True)

In [31]:
df_use['problem'] = None
df_use['cui'] = None

In [32]:
df_use

,id,EANDM_Suggested,EANDM_Final,CPT_Final,problem,cui
0,4858840,{99284=[]},99284,NaN,None,None
1,4858823,{99284=[]},99284,NaN,None,None
2,4858769,{99285=[25]},99285 = [25-Sig/Sep ID E&M Same Pro/Day],93010,None,None
4,4594032,"{99285=[25], 99284=[25]}",99284,NaN,None,None
5,4858762,{99285=[]},99284,NaN,None,None
...,...,...,...,...,...,...
2317,5747710,{99284=[]},99285,NaN,None,None
2318,5747680,{99285=[25]},99285,NaN,None,None
2319,5747641,{99284=[25]},99284 = [25-Sig/Sep ID E&M Same Pro/Day],41800,None,None
2320,5747690,{99285=[]},99285,NaN,None,None


In [33]:
# df_use[df_use.accountNumber==87257242]

In [34]:
file = open("/home/trishalaswain/data_for_analysis/SCP/Prod/2474/Output/out1.txt", "r")
account = ''

for i in file.readlines():
    problem_list = set()
    cui_set = set()
    
    if 'xml' in i:
        account_list = i.split("_")
        
        account = account_list[0]
        # print(account_list[0])
    else:   
        problem_cui_list = i.split("\t")
    #         problem_cui_list.remove('')
    #         problem_cui_list.remove('\n')

    #         print(problem_cui_list)
        for item in problem_cui_list:
            if(item == '' or item == '\n'):
                continue
            item = item.strip()
            problem = item.split("==")[0]
            cui = item.split("==")[1]
            
    #             print(problem)
    #             print(cui)
    
            problem_list.add(problem)
            cui_set.add(cui)
    
#         print(account)
#         print(problem_list)
#         print(cui_list)
#         print(problem_list)
#         print(cui_set)
#         print(df_use.loc[df_use["accountNumber"] == int(account)].shape)
#         print(account, print(type(account)))
        df_use.loc[df_use["id"] == int(account), "problem"] = ', '.join(problem_list)
        df_use.loc[df_use["id"] == int(account), "cui"] = ', '.join(cui_set)

In [35]:
# df_use[~df_use.cui.isna()]

In [37]:
temp = df_use[df_use.cui.fillna('').str.contains('100016270')]
temp[(temp.EANDM_Final.str.contains('99284')) & (temp.EANDM_Suggested.str.contains('99285')) & (~(temp.CPT_Final.str.contains('93010')))]

TypeError: bad operand type for unary ~: 'float'

In [11]:
# df_use.EANDM_Final.value_counts()

In [12]:
# df_use.EANDM_Suggested.value_counts()

In [13]:
df_use[(df_use.EANDM_Suggested.str.contains('99284')) & (df_use.EANDM_Final.str.contains('99285'))].shape

(83, 5)

In [14]:
df_use[(df_use.EANDM_Suggested.str.contains('99285')) 
      & (df_use.EANDM_Final.str.contains('99284'))].problem.value_counts()

Chest, Chest pain                                                                                                                                                                     9
Lower abdominal pain, abdominal pain, Lower                                                                                                                                           6
lumbar region, lumbar, Strain                                                                                                                                                         6
abdominal pain, Epigastric abdominal pain, Epigastric                                                                                                                                 5
pregnancy, Abdominal pain                                                                                                                                                             5
                                                                                

In [15]:
# from collections import defaultdict
# cui_count_99284 = defaultdict(int)

# for val in df_use[df_use.EANDM_Final.str.contains('99284')].cui:
#     for c in val:
#         cui_count_99284[c]+=1

# cui_count_99285 = defaultdict(int)
# for val in df_use[df_use.EANDM_Final.str.contains('99285')].cui:
#     for c in val:
#         cui_count_99285[c]+=1

In [16]:
# for key, val in sorted(cui_count_99285.items(), key=lambda item: item[1], reverse=True):
#     print(key, val, cui_count_99284[key])

In [17]:
from collections import defaultdict
def problemCount(code):
    problem_dict = {}
    problem_cui_dict = {}

    for index, row in df_use.iterrows():
        if row['EANDM_Final'].__contains__(code) and row['cui'] != None :
            cui_list = row['cui'].split(", ")
            for cui in cui_list:
                if cui not in problem_cui_dict.values():
                    problem_cui_dict[cui] = problem_cui_dict.get(cui,0)+ 1
                else:
                    problem_cui_dict[cui] = problem_cui_dict.get(cui)+ 1

    return problem_cui_dict

In [18]:
problem_dict_99284 = sorted(problemCount("99284").items(), key=lambda x:x[1], reverse=True)
problem_dict_99285 = sorted(problemCount("99285").items(), key=lambda x:x[1], reverse=True)

In [19]:
# df_problems = pd.DataFrame({'99284': problem_dict_99284, '99285': problem_dict_99285})

for key, value in problem_dict_99284:
    print(key + "\t" + str(value))

NO-CUI	867
205090	544
205091	499
205370	432
100017188	432
30193	377
439631	296
100003732	288
41912	230
120004548	201
100005126	177
3263722	162
12000504	158
180860	143
43246	135
31350	130
27497	126
441994	123
42965	115
100016591	113
16658	112
18681	108
100016270	107
9938	101
547045	97
24031	93
932	92
337196	91
29882	90
29877	90
677540	90
15967	88
347981	88
1280202	88
100016002	87
100016027	86
587267	86
238767	85
9450	83
1707968	80
26845	80
1282910	79
100005768	77
1281590	77
42740	75
817096	74
1283839	74
24090	74
1281562	73
1281583	73
443246	72
2111	72
16512	71
18965	71
1281584	71
120008524	70
833	69
35621	68
20538	68
7642	67
80194	66
439746	66
238656	64
700148	63
36396	63
13404	63
1281594	62
521421	60
1281591	59
231749	59
4604	58
43251	57
205064	57
100006742	57
15230	55
1963703	54
16659	53
42029	53
1267072	52
580209	52
2827407	52
262541	52
1278993	52
23216	51
19080	50
7859	50
38999	50
1285009	49
1278896	49
100007979	48
9763	47
521985	47
37011	46
205093	45
184567	44
2135307	44
221785	43


In [20]:
for key, value in problem_dict_99285:
    print(key + "\t" + str(value))

NO-CUI	867
100016270	546
205370	375
13404	286
120004548	227
439631	221
100017188	205
42965	196
27497	193
205090	179
30193	169
205091	156
100016002	130
100003732	119
441994	116
100003529	108
817096	106
20538	100
39070	99
42029	82
100016591	77
18965	77
4093	77
476207	75
547045	69
1279035	69
18681	69
9450	68
100000244	67
100002892	66
12000504	65
11175	64
1281594	64
32285	60
8035	60
30252	58
16658	54
2871	52
205246	51
1285009	48
100016027	47
587267	45
278061	45
120008524	45
2709248	44
1282910	42
3467	41
241863	40
1281590	40
38454	40
238767	39
15672	39
41912	39
1707968	38
100007979	38
1141948	37
41199	37
24117	36
120008357	36
16659	35
1281570	35
232493	35
16199	34
476206	34
7642	32
9938	32
392348	32
38999	32
220870	32
3263722	31
439746	31
17168	31
424000	31
438696	31
1704258	31
11878	31
443246	30
796494	30
337196	30
932	30
35621	29
20649	29
15967	29
4096	28
1281591	28
18801	28
205318	28
728899	28
4604	27
19080	27
100005126	27
1969	27
2609414	27
10692	26
581394	26
344304	26
36572	26
12000065